Importing libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
     

Loading data

In [2]:
filenames = os.listdir('./Data')
filenames

['April_2019.csv',
 'August_2019.csv',
 'December_2019.csv',
 'February_2019.csv',
 'January_2019.csv',
 'July_2019.csv',
 'June_2019.csv',
 'March_2019.csv',
 'May_2019.csv',
 'November_2019.csv',
 'October_2019.csv',
 'September_2019.csv']

In [4]:
entire_dataset = pd.DataFrame()
for file in filenames:
  df = pd.read_csv('./Data/'+file)
  entire_dataset = pd.concat([entire_dataset , df])

In [5]:
data = entire_dataset
data.sample(5)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
13009,308080,AA Batteries (4-pack),2,3.84,12/26/19 10:35,"294 North St, Seattle, WA 98101"
10648,172185,Wired Headphones,1,11.99,03/13/19 23:59,"261 Hill St, Dallas, TX 75001"
8711,287122,AA Batteries (4-pack),1,3.84,11/05/19 13:53,"895 Cherry St, San Francisco, CA 94016"
5426,300829,Lightning Charging Cable,1,14.95,12/23/19 18:02,"807 Highland St, Boston, MA 02215"
4248,213956,USB-C Charging Cable,1,11.95,06/12/19 19:58,"693 Dogwood St, Seattle, WA 98101"


In [6]:
data.shape

(186850, 6)